# Практическое задание 3

# Named Entity Recognition

Дата выдачи: <span style="color:red">__16 ноября 21:00__</span>.

Дедлайн: <span style="color:red">__01 декабря 22:00__</span>.

## Введение

### Постановка задачи

В этом задании вы будете решать задачу извлечения именованных сущностей (Named Entity Recognition) – одну из самых распространенных в NLP, наряду с задачей текстовой классификации.

Данная задача заключается в том, что нужно классифицировать каждое слово / токен на предмет того, является ли оно частью именованной сущности (сущность может состоять из нескольких слов / токенов) или нет.

Например, мы хотим извлечь имена и названия организаций. Тогда для текста

    Yan    Goodfellow  works  for  Google  Brain

модель должна извлечь следующую последовательность:

    B-PER  I-PER       O      O    B-ORG   I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система (*BIO*-разметка) введена, чтобы различать последовательные именованные сущности одного типа.
Существуют и другие типы разметок, например *BILUO*, но в рамках данного практического задания сфокусируемся имеено на *BIO*.

Решать NER задачу мы будем на датасете CoNLL-2003 с использованием рекуррентных сетей и моделей на базе архитектуры Transformer. Датасет CoNLL-2003 представлен в виде разметки **BIO**, где лейбл:
- *B-{label}* - начало сущности *{label}*;
- *I-{label}* - продолжение сущности *{label}*;
- *O* - отсутсвие сущности.

Здесь в качестве сущности *{label}* может выступать имя, географическое название или какой-то другой тип собственных имён. Подробнее с разметками можно ознакомится во вспомогательном ноутбуке.

### Библиотеки

Основные библиотеки:
 - [PyTorch](https://pytorch.org/)
 - [Transformers](https://github.com/huggingface/transformers)

### Данные

Данные лежат в архиве, который состоит из:

- *train.tsv* - обучающая выборка. В каждой строке записаны: <слово / токен>, <тэг слова / токена>

- *valid.tsv* - валидационная выборка, которую можно использовать для подбора гиперпарамеров и замеров качества. Имеет идентичную с train.tsv структуру.

- *test.tsv* - тестовая выборка, по которой оценивается итоговое качество. Имеет идентичную с train.tsv структуру.

Скачать данные можно здесь: [ссылка](https://github.com/valerapon/msu_task_3_ner)

In [1]:
!pip install numpy==1.23.5 scikit-learn==1.2.2 tensorboard==2.14.1 torch==2.1.0 tqdm==4.66.1 transformers==4.34.1

  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  Traceback (most recent call last):
    File "C:\Users\n1117\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\n1117\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\n1117\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
      backend = _build_backend()
                ^^^^^^^^^^^^^^^^
    File "C:\Users\n1117\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 77, in _build_backend
      obj = import_module(mod_path)
            ^^^^^^^^^^^^^^^^^^^^^

In [2]:
import random
from collections import Counter, defaultdict, namedtuple
from typing import Tuple, List, Dict, Any

import torch
import numpy as np

from tqdm import trange
from tqdm.notebook import tqdm

In [3]:
import os
os.chdir("C:\\Users\\n1117\\CODES_NOTE\\ML_NOTE\\HW_7sem\\4hw")

Зафиксируем seed (значение 42) для воспроизводимости результатов (желательно делать **всегда**!):

In [4]:
def set_global_seed(seed: int) -> None:
    """
    Set global seed for reproducibility.
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


set_global_seed(42)

Проинициализируем device (CPU / GPU) на котором будем работать (желательно **GPU**):

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Часть 1. Подготовка данных (4 балла)

Первым делом нам нужно считать данные. Давайте напишем функцию, которая на вход принимает путь до одного из conll-2003 файла и возвращает два списка:
- список списков слов / токенов;
- список списков тегов, который соответствует собранному списку слов / токенов.

Также функция на вход принимает булевую переменную *lowercase*, которая задает чувствительность к регистру. Далее будем всё приводить к нижнему регистру (`lower=True`).

P.S. Стоит держать в голове, что в некоторых ситуациях верхний регистр помогает выявлять именованные сущности. Например, у вас нет мощностей, чтобы запускать сложные модели, а задачу решать нужно быстро. В этом случае эвристическое правило: "Большая буква в слове = именнованная сущность" может вам помочь. Или у вас есть огромные корпусы данных, которые позволяют сохранять исходное разнообразие слов.

**Задание. Реализуйте функцию read_conll2003.** **<font color='red'>(1 балл)</font>**

<details>
<summary> Подсказка </summary>

*Предложения разделены пустой строкой, в конце файла также пустая строка, но не забывайте про символ `\n`.*

</details>


In [6]:
def read_conll2003(
    path: str,
    lower: bool = True,
) -> Tuple[List[List[str]], List[List[str]]]:
    """
    Prepare data in CoNNL like format.

    Args:
        path: The path to the file (str).
        lower:  Reduce text to lowercase (bool).

    Returns:
        Function returns pair (token_seq, label_seq).
        token_seq: The list of lists. Each internal list is
            a sentence converted into tokens.
        label_seq: The list of lists. All internal lists
            contain tags corresponding to tokens from token_seq.

    """

    token_seq: List[List[str]] = []
    label_seq: List[List[str]] = []
    current_sentence = []
    current_tags = []

    with open(path, "r") as text_inp_file:
         for line in text_inp_file:
            line = line.strip()
            if line:
                parts = line.split()
                token = parts[0]
                tag = parts[1]
                if lower:
                    token = token.lower()
                current_sentence.append(token)
                current_tags.append(tag)
            else:
                if current_sentence:
                    token_seq.append(current_sentence)
                    label_seq.append(current_tags)
                    current_sentence = []
                    current_tags = []


    return token_seq, label_seq

Считаем все три файла:
- *train.tsv*
- *valid.tsv*
- *test.tsv*

In [7]:
train_token_seq, train_label_seq = read_conll2003("data/train.txt")
valid_token_seq, valid_label_seq = read_conll2003("data/valid.txt")
test_token_seq, test_label_seq = read_conll2003("data/test.txt")

Посмотрим на то, что мы получили:

In [8]:
for token, label in zip(train_token_seq[0], train_label_seq[0]):
    print(f"{token}\t{label}")

eu	B-ORG
rejects	O
german	B-MISC
call	O
to	O
boycott	O
british	B-MISC
lamb	O
.	O


In [9]:
for token, label in zip(valid_token_seq[0], valid_label_seq[0]):
    print(f"{token}\t{label}")

cricket	O
-	O
leicestershire	B-ORG
take	O
over	O
at	O
top	O
after	O
innings	O
victory	O
.	O


In [10]:
for token, label in zip(test_token_seq[0], test_label_seq[0]):
    print(f"{token}\t{label}")

soccer	O
-	O
japan	B-LOC
get	O
lucky	O
win	O
,	O
china	B-PER
in	O
surprise	O
defeat	O
.	O


In [11]:
assert len(train_token_seq) == len(train_label_seq), "Длины тренировочных token_seq и label_seq не совпадают, ошибка в функции read_conll2003"
assert len(valid_token_seq) == len(valid_label_seq), "Длины валидационных token_seq и label_seq не совпадают, ошибка в функции read_conll2003"
assert len(test_token_seq) == len(test_label_seq), "Длины тестовых token_seq и label_seq не совпадают, ошибка в функции read_conll2003"

assert train_token_seq[0] == ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], "Ошибка в тренировочном token_seq"
assert train_label_seq[0] == ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'], "Ошибка в тренировочном label_seq"

assert valid_token_seq[0] == ['cricket', '-', 'leicestershire', 'take', 'over', 'at', 'top', 'after', 'innings', 'victory', '.'], "Ошибка в валидационном token_seq"
assert valid_label_seq[0] == ['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], "Ошибка в валидационном label_seq"

assert test_token_seq[0] == ['soccer', '-', 'japan', 'get', 'lucky', 'win', ',', 'china', 'in', 'surprise', 'defeat', '.'], "Ошибка в тестовом token_seq"
assert test_label_seq[0] == ['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O'], "Ошибка в тестовом label_seq"

print("Тесты пройдены!")

Тесты пройдены!


### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения:
- {**token**}→{**token_idx**}: соответствие между словом / токеном и индексом строки в *embedding* матрице (начинается с 0);
- {**label**}→{**label_idx**}: соответствие между тегом и уникальным индексом (начинается с 0).

Теперь нам необходимо реализовать две функции:
- *get_token2idx*
- *get_label2idx*

которые будут возвращать соответствующие словари (*token2idx* и *label2idx*).

Словарь *token2idx* должен содержать специальные токены, которые нужно добавить самим:
- `<PAD>` - спецтокен для паддинга (отступа), так как мы собираемся обучать модели батчами. Токен `<PAD>` нужен для выравнивания предложений по длине, когда их будем помещать в один батч. Чаще всего предложения дополняются с конца;
- `<UNK>` - спецтокен для обработки слов / токенов, которых нет в словаре (актуально для инференса).

Давайте для удобства дадим токену `<PAD>` индекс `0`, а токену `<UNK>` индекс `1`.

В функцию *get_token2idx* также необходимо добавить параметр *min_count*, который будет включать только слова, превышающие определенную частоту.

Для начала посмотрим, а сколько вообще уникальных слов в обучающих данных и число их вхождений:

In [12]:
token_counter = Counter([token for sentence in train_token_seq for token in sentence])
print(*token_counter.most_common(10), sep='\n')
print(f"Количество уникальных слов в тренировочном датасете: {len(token_counter)}")
print(f"Количество слов встречающихся только один раз в тренировочном датасете: {len([token for token, cnt in token_counter.items() if cnt == 1])}")

('the', 8390)
('.', 7374)
(',', 7290)
('of', 3815)
('in', 3621)
('to', 3424)
('a', 3199)
('and', 2872)
('(', 2861)
(')', 2861)
Количество уникальных слов в тренировочном датасете: 21010
Количество слов встречающихся только один раз в тренировочном датасете: 10060


Как мы видим, у нас есть много слов, которые в обучении встречаются только один раз. Очевидно, что выучиться по ним у нас не получиться, мы только переобучимся, поэтому давайте выкинем такие слова при формировании нашего словаря, задав параметр функции `min_count=2`.

На этом этапе можно применять различные методы сокращения размера словаря, преобразовывая разные словоформы одного слова в один токен: стемминг или лемматизация. В текущей задаче мы это опустим, но в некоторых ситуациях это бывает полезно.

**Задание. Реализуйте функции get_token2idx и get_label2idx.** **<font color='red'>(1 балл)</font>**

<details>
<summary> Подсказка №1 </summary>

*Не забудьте, что у \<PAD\> индекс 0, а у \<UNK\> индекс 1.*

</details>

<details>
<summary> Подсказка №2 </summary>

*Лучше всего словари token2idx и label2idx собирать с помощью token2cnt в get_token2idx и label_list в get_label2idx соответственно.*

</details>

<details>
<summary> Подсказка №3 </summary>

*В label_list в get_label2idx лежат отсортированные по возрастанию тэги, за исключением тэга 'O' – он идет первый (индекс 0). Этот порядок нужно сохранить при индексации в label2idx.*

</details>

In [13]:
def get_token2idx(
    token_seq: List[List[str]],
    min_count: int,
) -> Dict[str, int]:
    """
    Get mapping from tokens to indices to use with Embedding layer.

    Args:
        token_seq: The list of lists. Each internal list (sentence)
            consists of tokens.
        min_count:  The minimum number of repetitions of
            a token in the corpus.

    Returns:
        Function returns mapping from token to id.
        token2idx: The mapping from token
            to id without "rare" words.

    """

    token2idx: Dict[str, int] = {'<PAD>': 0, '<UNK>': 1}
    token2cnt = Counter([token for sentence in token_seq for token in sentence])

    idx = 2
    for token, count in token2cnt.items():
        if count >= min_count:
            token2idx[token] = idx
            idx += 1

    return token2idx

In [14]:
token2idx = get_token2idx(train_token_seq, min_count=2)

In [15]:
def get_label2idx(label_seq: List[List[str]]) -> Dict[str, int]:
    """
    Get mapping from labels to indices.

    Args:
        label_seq: The list of lists. Each internal list (sentence)
            consists of labels.

    Returns:
        Function returns mapping from label to id.
        label2idx: The mapping from label to id.

    """

    label2idx: Dict[str, int] = {}
    label_list = set(label for sentence in label_seq for label in sentence)
    label_list = sorted(label_list, key=lambda x: 'A' if x == 'O' else x)

    for idx, label in enumerate(label_list):
        label2idx[label] = idx

    return label2idx

In [16]:
label2idx = get_label2idx(train_label_seq)

Посмотрим на то, что мы получили:

In [17]:
for token, idx in list(token2idx.items())[:10]:
    print(f"{token}\t{idx}")

<PAD>	0
<UNK>	1
eu	2
german	3
call	4
to	5
boycott	6
british	7
lamb	8
.	9


In [18]:
for label, idx in label2idx.items():
    print(f"{label}\t{idx}")

O	0
B-LOC	1
B-MISC	2
B-ORG	3
B-PER	4
I-LOC	5
I-MISC	6
I-ORG	7
I-PER	8


In [19]:
assert len(get_token2idx(train_token_seq, min_count=1)) == 21012, "Ошибка в длине словаря, скорее всего неверно реализован min_count"
assert len(token2idx) == 10952, "Неправильная длина token2idx, скорее всего неверно реализован min_count"
assert len(label2idx) == 9, "Неправильная длина label2idx"

assert list(token2idx.items())[:10] == [('<PAD>', 0), ('<UNK>', 1), ('eu', 2), ('german', 3), ('call', 4), ('to', 5), ('boycott', 6), ('british', 7), ('lamb', 8), ('.', 9)], "Неправильно сформированный token2idx"
assert label2idx == {'O': 0, 'B-LOC': 1, 'B-MISC': 2, 'B-ORG': 3, 'B-PER': 4, 'I-LOC': 5, 'I-MISC': 6, 'I-ORG': 7, 'I-PER': 8}, "Неправильно сформированный label2idx"

print("Тесты пройдены!")

Тесты пройдены!


### Подготовка датасета и загрузчика

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины.

Из предыдущего практического задания вы должны знать о `Dataset`'е (`torch.utils.data.Dataset`) – структура данных, которая хранит и может по индексу отдавать данные для обучения. Датасет должен наследоваться от стандартного PyTorch класса Dataset и переопределять методы `__len__` и `__getitem__`.

Метод `__getitem__` должен возвращать индексированную последовательность и её теги.

**Не забудьте** про `<UNK>` спецтокен для неизвестных слов!
    
Давайте напишем кастомный датасет под нашу задачу, который на вход (метод `__init__`) будет принимать:
- *token_seq* – список списков слов / токенов;
- *label_seq* – список списков тегов;
- *token2idx* – отображение токена в индекс;
- *label2idx* – отображение тега в индекс.

и возвращать из метода `__getitem__` два Int64 тензора (`torch.LongTensor`) из индексов слов / токенов в сэмпле и индексов соответвующих тегов:

**Задание. Реализуйте класс датасета NERDataset.** **<font color='red'>(1 балл)</font>**

<details>
<summary> Подсказка </summary>

*Для понимания, зачем нужен вообще этот Dataset. У нас по факту есть "сырые" последовательности token_seq и label_seq, которые по индексу будут возвращать и нужный набор токенов, и соответствующий список тегов. Но для обучения это неудобно, потому что каждый раз нам необходимо конвертировать токены и теги в индексы. Инструмент Dataset нужен для того, чтобы, не задумываясь, извлекать элементы в нужном формате автоматически, как из массива.*

</details>

In [20]:
class NERDataset(torch.utils.data.Dataset):
    """
    PyTorch Dataset for NER.
    """

    def __init__(
        self,
        token_seq: List[List[str]],
        label_seq: List[List[str]],
        token2idx: Dict[str, int],
        label2idx: Dict[str, int],
    ) -> None:
        """
        Constructor of NERDataset class.

        Args:
            token_seq: The list of lists. Each internal list (sentence)
                consists of tokens.
            label_seq: The list of lists. Each internal list (sentence)
                consists of labels.
            token2idx: The mapping from token to id.
            label2idx: The mapping from label to id.

        Returns:
            None

        """
        self.token2idx = token2idx
        self.label2idx = label2idx

        self.token_seq = [self.process_tokens(tokens, token2idx) for tokens in token_seq]
        self.label_seq = [self.process_labels(labels, label2idx) for labels in label_seq]

    def __len__(self) -> int:
        """
        Get dataset size.

        Args:

        Returns:
            The method returns the length of the dataset.

        """
        return len(self.token_seq)

    def __getitem__(
        self,
        idx: int,
    ) -> Tuple[torch.LongTensor, torch.LongTensor]:
        """
        Get an element from a dataset by index.
        Recomendation: use LongTensor.

        Args:
            idx: Index of element (int).

        Returns:
            The method returns required element. From
            self.token_seq and self.label_seq.

        """
        token_ids = torch.LongTensor(self.token_seq[idx])
        label_ids = torch.LongTensor(self.label_seq[idx])
        
        return token_ids, label_ids

    @staticmethod
    def process_tokens(
        tokens: List[str],
        token2idx: Dict[str, int],
        unk: str = "<UNK>",
    ) -> List[int]:
        """
        Transform list of tokens into list of tokens' indices.

        Args:
            tokens: The list (sentence) of tokens.
            token2idx: The mapping from token to id.
            unk: Name of <UNK> token (str).

        Returns:
            token_ids: The list of indices. Each index
                corresponds to a token from the tokens.

        """
        token_ids: List[int] = []

        for token in tokens:
            token_ids.append(token2idx.get(token, token2idx[unk]))

        return token_ids

    @staticmethod
    def process_labels(
        labels: List[str],
        label2idx: Dict[str, int],
    ) -> List[int]:
        """
        Transform list of labels into list of labels' indices.

        Args:
            labels: The list of labels.
            label2idx: The mapping from label to id.

        Returns:
            label_ids: The list of indices. Each index
                corresponds to a label from the labels.

        """
        label_ids: List[int] = []

        for label in labels:
            label_ids.append(label2idx[label])

        return label_ids

Создадим три датасета:
- *train_dataset*
- *valid_dataset*
- *test_dataset*

In [21]:
train_dataset = NERDataset(
    token_seq=train_token_seq,
    label_seq=train_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
valid_dataset = NERDataset(
    token_seq=valid_token_seq,
    label_seq=valid_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
test_dataset = NERDataset(
    token_seq=test_token_seq,
    label_seq=test_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)

Посмотрим на то, что мы получили:

In [22]:
train_dataset[0]

(tensor([2, 1, 3, 4, 5, 6, 7, 8, 9]), tensor([3, 0, 2, 0, 0, 0, 2, 0, 0]))

In [23]:
valid_dataset[0]

(tensor([1737,  571, 1777,  197,  687,  145,  349,  111, 1819, 1558,    9]),
 tensor([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]))

In [24]:
test_dataset[0]

(tensor([1516,  571, 1434, 1729, 4893, 2014,   67,  310,  215, 3157, 3139,    9]),
 tensor([0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0]))

In [25]:
assert len(train_dataset) == 14986, "Неправильная длина train_dataset"
assert len(valid_dataset) == 3465, "Неправильная длина valid_dataset"
assert len(test_dataset) == 3683, "Неправильная длина test_dataset"

assert torch.equal(train_dataset[0][0], torch.tensor([2,1,3,4,5,6,7,8,9])), "Неправильно сформированный train_dataset"
assert torch.equal(train_dataset[0][1], torch.tensor([3,0,2,0,0,0,2,0,0])), "Неправильно сформированный train_dataset"

assert torch.equal(valid_dataset[0][0], torch.tensor([1737,571,1777,197,687,145,349,111,1819,1558,9])), "Неправильно сформированный valid_dataset"
assert torch.equal(valid_dataset[0][1], torch.tensor([0,0,3,0,0,0,0,0,0,0,0])), "Неправильно сформированный valid_dataset"

assert torch.equal(test_dataset[0][0], torch.tensor([1516,571,1434,1729,4893,2014,67,310,215,3157,3139,9])), "Неправильно сформированный test_dataset"
assert torch.equal(test_dataset[0][1], torch.tensor([0,0,1,0,0,0,0,4,0,0,0,0])), "Неправильно сформированный test_dataset"

print("Тесты пройдены!")

Тесты пройдены!


Для того, чтобы дополнять последовательности паддингом, будем использовать параметр `collate_fn` класса `DataLoader`.

Принимая последовательность пар тензоров для предложений и тегов, необходимо дополнить все последовательности до последовательности максимальной длины в батче.

Используйте для дополнения спецтокен `<PAD>` для последовательностей слов / токенов и -1 для последовательностей тегов.

**hint**: удобно использовать метод **torch.nn.utils.rnn**. Обратите особое внимание на параметр *batch_first*

`Collator` можно реализовать двумя способами:
- класс с методом `__call__`
- функцию

Мы пойдем первым путем.

Инициализировать экземпляр класса `Collator` (метод `__init__`) с помощью двух параметров:
- id `<PAD>` спецтокена для последовательностей слов / токенов
- id `<PAD>` спецтокена для последовательностей тегов (значение -1)

Метод `__call__` на вход принимает батч, а именно список кортежей того, что нам возвращается из датасета. В нашем случае это список кортежей двух int64 тензоров - `List[Tuple[torch.LongTensor, torch.LongTensor]]`.

На выходе мы хотим получить два тензора:
- западденные индексы слов / токенов
- западденные индексы тегов
    
P.S. `<PAD>` значение нужно для того, чтобы при подсчете лосса легко отличать западдированные токены от других. Можно использовать параметр *ignore_index* при инициализации лосса.

**Задание. Реализуйте класс коллатора NERCollator.** **<font color='red'>(1 балл)</font>**

<details>
<summary> Пояснение </summary>

*Чтобы ускорить взаимодействие с нейронными сетями, им на вход подаются не одна пара (объект, ответ), а несколько. Они объединяются в одну связку, которую называют батч. В нашем случае объекты -- это последовательности слов, которые имеют разную длину. Мы не можем в одном pytorch.tensor (или numpy.array) хранить последовательности разных длин, поэтому требуется их привести к одной (например, к длине максимальной последовательности), чем коллатор и занимается.*

</details>

In [26]:
from torch.nn.utils.rnn import pad_sequence

class NERCollator:
    """
    Collator that handles variable-size sentences.
    """

    def __init__(
        self,
        token_padding_value: int,
        label_padding_value: int,
    ):
        self.token_padding_value = token_padding_value
        self.label_padding_value = label_padding_value

    def __call__(
        self,
        batch: List[Tuple[torch.LongTensor, torch.LongTensor]],
    ) -> Tuple[torch.LongTensor, torch.LongTensor]:
        """
        The method appends <PAD> token id to tensor with
        token ids and -1 to tensor with labels.
        The function torch.nn.utils.rnn.pad_sequence is useful for padding. Link:
        https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html

        Args:
            batch: The list of tuples. Each tuple conists of the tensor with token ids and
                the tensor with label ids.

        Returns:
            (tokens, labels): a pair of tensors with sizes: (batch_size, sentence_len).

        """
        tokens, labels = zip(*batch)

        tokens = pad_sequence(tokens, batch_first=True, padding_value=self.token_padding_value)
        labels = pad_sequence(labels, batch_first=True, padding_value=self.label_padding_value)

        return tokens, labels

In [27]:
collator = NERCollator(
    token_padding_value=token2idx["<PAD>"],
    label_padding_value=-1,
)

Теперь всё готово, чтобы задать `DataLoader`'ы:

In [28]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collator,
)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)

Посмотрим на то, что мы получили:

In [29]:
tokens, labels = next(iter(train_dataloader))

tokens = tokens.to(device)
labels = labels.to(device)

In [30]:
tokens

tensor([[7796, 1162, 2553, 7237, 1342,    0,    0,    0,    0,    0],
        [ 125, 1167,    1,   67, 1349,  489, 1215, 1364, 1365, 1366]],
       device='cuda:0')

In [31]:
labels

tensor([[ 3,  0,  3,  7,  0, -1, -1, -1, -1, -1],
        [ 0,  4,  8,  0,  1,  0,  0,  0,  0,  0]], device='cuda:0')

In [32]:
train_tokens, train_labels = next(iter(
    torch.utils.data.DataLoader(
        train_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(train_tokens, torch.tensor([[ 2,  1,  3,  4,  5,  6,  7,  8,  9], [10, 11,  0,  0,  0,  0,  0,  0,  0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(train_labels, torch.tensor([[ 3,  0,  2,  0,  0,  0,  2,  0,  0], [ 4,  8, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

valid_tokens, valid_labels = next(iter(
    torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(valid_tokens, torch.tensor([[ 1737,   571,  1777,   197,   687,   145,   349,   111,  1819,  1558, 9], [  248, 10679,     0,     0,     0,     0,     0,     0,     0,     0,    0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(valid_labels, torch.tensor([[ 0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0], [ 1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

test_tokens, test_labels = next(iter(
    torch.utils.data.DataLoader(
        test_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(test_tokens, torch.tensor([[1516,  571, 1434, 1729, 4893, 2014,   67,  310,  215, 3157, 3139,    9], [   1,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(test_labels, torch.tensor([[ 0,  0,  1,  0,  0,  0,  0,  4,  0,  0,  0,  0], [ 4,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

print("Тесты пройдены!")

Тесты пройдены!


## Часть 2. BiLSTM-теггер (6 баллов)

Определите архитектуру сети, используя библиотеку PyTorch.

Ваша архитектура в этом пункте должна соответствовать стандартному теггеру:
* Embedding слой на входе
* LSTM (однонаправленный или двунаправленный) слой для обработки последовательности
* Dropout (заданный отдельно или встроенный в LSTM) для уменьшения переобучения
* Linear слой на выходе

Для обучения сети используйте поэлементную кросс-энтропийную функцию потерь.

**Обратите внимание**, что `<PAD>` токены не должны учавствовать в подсчёте функции потерь. В качестве оптимизатора рекомендуется использовать Adam. Для получения значений предсказаний по выходам модели используйте функцию `argmax`.

**Задание. Реализуйте класс модели BiLSTM.** **<font color='red'>(2 балл)</font>**

<details>
<summary> Подсказка №1 </summary>

*Помните, что следуется явно указывать параметр `batch_first=True`.*

</details>

<details>
<summary> Подсказка №2 </summary>

*Число входных признаков `in_features` в линейном слое зависим от типа LSTM: однонаправленная или двунаправленая. В первом случае выходом i-го блока является скрытое состояние $h_i$, которое имеет размер `hidden_size`, и соответственно, `in_features=hidden_size`. А в случае двунаправленной сети вход представим в виде конкатенации скрытых состояний из разных уровней сети: $[h_i^1, h_i^2]$, здесь `in_features=2 * hidden_size`.*

</details>

<details>
<summary> Рекомендация </summary>

*Обратите внимание на метод `BiLSTM.forward`, реализовывать самостоятельно его не нужно. В нем используется функция `pack_padded_sequence` и обратная ей `pad_packed_sequence`. Это удобный инструмент для фильтрации входных последовательностей от токенов отступа.*

</details>

In [33]:
from torch.nn import Embedding, LSTM, Linear


class BiLSTM(torch.nn.Module):
    """
    Bidirectional LSTM architecture.
    """

    def __init__(
        self,
        num_embeddings: int,
        embedding_dim: int,
        hidden_size: int,
        num_layers: int,
        dropout: float,
        bidirectional: bool,
        n_classes: int,
        token_padding_value: int,
        max_norm: float,
    ) -> None:
        """
        BiLSTM class constructor. The method contains a description
        of the network layers and their parameters.

        Args:
            num_embeddings: The number of unique words (tokens) in the
                dictionary, or the size of the dictionary (int)
            embedding_dim:  Embedding word size (size)
            hidden_size: The size of the hidden state (h_n) in the LSTM network (int)
            num_layers: The total number of blocks in an LSTM network (int)
            dropout: dropout parameter in LSTM layers (float)
            bidirectional: a parameter that controls whether the LSTM is
                unidirectional (one direction) or bidirectional (bool)
            n_classes: the number of classes in a problem being solved (int)

        Returns:
            None

        Tips:
            - do not forget to specify batch_first=True
            - control the size of the in_features in linear
                layer depending on the value of 'bidirectional'
        """
        super().__init__()

        self.token_padding_value = token_padding_value
        self.embedding = None # Embedding layer
        self.rnn = None # LSTM layer
        self.head = None # Linear layer

        self.embedding = Embedding(num_embeddings, embedding_dim, padding_idx=token_padding_value)

        self.rnn = LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            bidirectional=bidirectional,
            batch_first=True
        )

        in_features = hidden_size * 2 if bidirectional else hidden_size
        self.head = Linear(in_features, n_classes)

    def forward(self, tokens: torch.LongTensor) -> torch.Tensor:
        """
        Applying neural network layers to input 'tokens'.

        Args:
            tokens: the input tensor with tokens ids (batch_size, sequence_len)

        Returns:
            logits: the scores issued by the model (batch_size, num_classes, sequence_len)
        """
        embed = self.embedding(tokens)

        # Используем специальную функцию pack_padded_sequence для того, чтобы получить
        # структуру PackedSequence, которая не учитывать паддинг при проходе rnn.
        # lengths -- длины исходных исходных последовательностей в батче,
        # без учёта сдвига
        lengths = (tokens != self.token_padding_value).sum(dim=1).detach().cpu()
        packed_embed = torch.nn.utils.rnn.pack_padded_sequence(
            input=embed,
            lengths=lengths,
            batch_first=True,
            enforce_sorted=False,
        )

        # Используем специальную функцию pad_packed_sequence для того, чтобы получить
        # тензор из PackedSequence. Операция является обратной к pack_padded_sequence
        packed_rnn_output, _ = self.rnn(packed_embed)
        rnn_output, _ = torch.nn.utils.rnn.pad_packed_sequence(
            sequence=packed_rnn_output,
            batch_first=True
        )

        logits = self.head(rnn_output)
        logits = logits.transpose(1, 2)
        return logits

In [34]:
model = BiLSTM(
    num_embeddings=len(token2idx),
    embedding_dim=300,
    hidden_size=256,
    num_layers=1,
    dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
    token_padding_value=token2idx["<PAD>"],
    max_norm=None,
).to(device)

In [35]:
model

BiLSTM(
  (embedding): Embedding(10952, 300, padding_idx=0)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (head): Linear(in_features=512, out_features=9, bias=True)
)

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

In [37]:
outputs = model(tokens)

In [38]:
assert outputs.shape == torch.Size([2, 9, 10])
assert 2 < criterion(outputs, labels) < 3

print("Тесты пройдены!")

Тесты пройдены!


### Эксперименты

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1-macro мере на валидационной и тестовой выборках было не меньше **0.76**.

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

In [39]:
# Создадим SummaryWriter для эксперимента с BiLSTMModel
# для отслеживания процесса обучения нейронной сети

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=f"logs/BiLSTMModel")

**Задание. Реализуйте функцию подсчета метрик compute_metrics.** **<font color='red'>(1 балл)</font>**

<details>
<summary> Подсказка №1 </summary>

*Модель выдает логиты, или скоры, для каждого токена по каждому классу. Для подсчета метрик нужно, подобно максимизации вероятности, каждому токену входной последовательности определять класс с максимальный скором. Пример: токен `X` получил скоры (выход модели) для четырех классов `[0.5, 10.2, -13,9, 5,5]` соответственно, следовательно, нам необходимо определять для токена `X` класс № 1 (нумерация с нуля, `score=10.2`) как наиболее "вероятный".*

</details>

<details>
<summary> Подсказка №2 </summary>

*Входные тензоры необходимо перевести на CPU (если они на GPU), конвертировать в numpy-массив и для простоты вытянуть в вектор с помощью функции `flatten`.*

</details>

<details>
<summary> Подсказка №3 </summary>

*Не забудьте отфильтровать `<PAD>` токен.*

</details>

In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def compute_metrics(
    outputs: torch.Tensor,
    labels: torch.LongTensor,
) -> Dict[str, float]:
    """
    Compute NER metrics.

    Args:
        outputs: the model outputs (batch_size, num_classes, sequence_len)
        labels: the correct classes (batch_size, sequence_len)

    Returns:
        metrics: mapping metric names to their corresponding values
    """

    metrics = {}
    y_true = None
    y_pred = None

    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    y_pred = outputs.argmax(axis=1)
    valid_mask = labels != -1
    y_true = labels[valid_mask]
    y_pred = y_pred[valid_mask]
    
    metrics['accuracy'] = accuracy_score(
        y_true=y_true,
        y_pred=y_pred,
    )

    for metric_func in [precision_score, recall_score, f1_score]:
        metric_name = metric_func.__name__.split('_')[0]
        for average_type in ["micro", "macro", "weighted"]:
            metrics[metric_name + '_' + average_type] = metric_func(
                y_true=y_true,
                y_pred=y_pred,
                average=average_type,
                zero_division=0,
            )

    return metrics

In [41]:
compute_metrics(
    outputs=outputs,
    labels=labels,
)

{'accuracy': 0.06666666666666667,
 'precision_micro': 0.06666666666666667,
 'precision_macro': 0.0625,
 'precision_weighted': 0.3,
 'recall_micro': 0.06666666666666667,
 'recall_macro': 0.013888888888888888,
 'recall_weighted': 0.06666666666666667,
 'f1_micro': 0.06666666666666667,
 'f1_macro': 0.022727272727272728,
 'f1_weighted': 0.1090909090909091}

**Задание. Реализуйте функции обучения и тестирования train_epoch и evaluate_epoch.** **<font color='red'>(2 балла)</font>**

<details>
<summary> Подсказка №1 </summary>

*Почти всегда шаг обучения модели остается неизменным. Нужно выполнить последовательность действий: обнулить градиент, получить выходы модели, посчитать лосс, посчитать новые градиенты через `.backward()`, сделать шаг оптимизатора.*

</details>

<details>
<summary> Подсказка №2 </summary>

*При evaluate-этапе никакие градиенты не вычисляются, потому нет необходимости ни их в обнулении, ни в оптимизации по ним.*

</details>

<details>
<summary> Пояснение </summary>

*В PyTorch градиенты аккумулируются, их нужно сбрасывать через `optimizer.zero_grad()` перед каждой новой итерацией, чтобы исключить влияние градиента с предыдущих итераций на шаг текущего.*

</details>

In [42]:
def train_epoch(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
    model_type: str,
) -> None:
    """
    One training cycle (loop).

    Args:
        model: BiLSTM model
        dataloader: Dataloader with train data
        optimizer: an algorithm for model optimization
        criterion: the loss function
        writer: a tool for logging the learning process
        device: the device on which the model will work
        epoch: the total number of epochs

    Returns:
        None
    """

    model.train()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)

    for i, (tokens, labels) in tqdm(
        enumerate(dataloader),
        total=len(dataloader),
        desc="loop over train batches",
    ):

        tokens, labels = tokens.to(device), labels.to(device)

        outputs = None
        loss = None

        if model_type == 'BiLSTM':
            logits = model.forward(tokens)
            outputs = logits
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
        elif model_type == 'Transformer':
            logits = model.forward(**tokens).logits
            loss = criterion(logits.transpose(1, 2), labels)
            loss.backward()
            optimizer.step()
        else:
            raise ValueError('Use \'BiLSTM\' or \'Transformer\' model_type.')

        epoch_loss.append(loss.item())
        writer.add_scalar(
            tag="batch loss / train",
            scalar_value=loss.item(),
            global_step=epoch * len(dataloader) + i,
        )

        with torch.no_grad():
            model.eval()
            if model_type == 'BiLSTM':
                outputs_inference = model(tokens)
            elif model_type == 'Transformer':
                outputs_inference = model(**tokens)["logits"].transpose(1, 2)
            else:
                raise ValueError('Use \'BiLSTM\' or \'Transformer\' model_type.')
            model.train()

        batch_metrics = compute_metrics(
            outputs=outputs_inference,
            labels=labels,
        )

        for metric_name, metric_value in batch_metrics.items():
            batch_metrics_list[metric_name].append(metric_value)
            writer.add_scalar(
                tag=f"batch {metric_name} / train",
                scalar_value=metric_value,
                global_step=epoch * len(dataloader) + i,
            )

    avg_loss = np.mean(epoch_loss)
    print(f"Train loss: {avg_loss}\n")
    writer.add_scalar(
        tag="loss / train",
        scalar_value=avg_loss,
        global_step=epoch,
    )

    for metric_name, metric_value_list in batch_metrics_list.items():
        metric_value = np.mean(metric_value_list)
        print(f"Train {metric_name}: {metric_value}\n")
        writer.add_scalar(
            tag=f"{metric_name} / train",
            scalar_value=metric_value,
            global_step=epoch,
        )

In [76]:
def evaluate_epoch(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
    model_type: str,
) -> None:
    """
    One evaluation cycle (loop).

    Args:
        model: BiLSTM model
        dataloader: Dataloader with data for evaluation
        criterion: a loss function
        writer: a tool for logging the learning process
        device: the device on which the model will work
        epoch: the total number of epochs

    Returns:
        None
    """

    model.eval()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)

    with torch.no_grad():

        for i, (tokens, labels) in tqdm(
            enumerate(dataloader),
            total=len(dataloader),
            desc="loop over test batches",
        ):

            tokens, labels = tokens.to(device), labels.to(device)

            if model_type == 'BiLSTM':
                logits = model.forward(tokens)
                outputs = logits
                loss = criterion(logits, labels)
            elif model_type == 'Transformer':
                logits = model.forward(**tokens).logits
                outputs = logits.transpose(1, 2)
                loss = criterion(logits.transpose(1, 2), labels)
            else:
                raise ValueError('Use \'BiLSTM\' or \'Transformer\' model_type.')

            epoch_loss.append(loss.item())
            writer.add_scalar(
                tag="batch loss / test",
                scalar_value=loss.item(),
                global_step=epoch * len(dataloader) + i,
            )

            batch_metrics = compute_metrics(
                outputs=outputs,
                labels=labels,
            )

            for metric_name, metric_value in batch_metrics.items():
                batch_metrics_list[metric_name].append(metric_value)
                writer.add_scalar(
                    tag=f"batch {metric_name} / test",
                    scalar_value=metric_value,
                    global_step=epoch * len(dataloader) + i,
                )

        avg_loss = np.mean(epoch_loss)
        print(f"Test loss:  {avg_loss}\n")
        writer.add_scalar(
            tag="loss / test",
            scalar_value=avg_loss,
            global_step=epoch,
        )

        for metric_name, metric_value_list in batch_metrics_list.items():
            metric_value = np.mean(metric_value_list)
            print(f"Test {metric_name}: {metric_value}\n")
            writer.add_scalar(
                tag=f"{metric_name} / test",
                scalar_value=np.mean(metric_value),
                global_step=epoch,
            )

In [44]:
def train(
    n_epochs: int,
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    valid_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
    model_type: str,
) -> None:
    """
    Training loop.

    Args:
        n_epochs: the total number of epochs in training
        model: BiLSTM model
        train_dataloader:  Dataloader with train data
        valid_dataloader: Dataloader with data for evaluation
        optimizer: an algorithm for model optimization
        criterion: a loss function
        writer: a tool for logging the learning process
        device: the device on which the model will work

    Returns:
        None
    """

    for epoch in range(n_epochs):

        print(f"Epoch [{epoch+1} / {n_epochs}]\n")

        train_epoch(
            model=model,
            dataloader=train_dataloader,
            optimizer=optimizer,
            criterion=criterion,
            writer=writer,
            device=device,
            epoch=epoch,
            model_type=model_type,
        )
        evaluate_epoch(
            model=model,
            dataloader=valid_dataloader,
            criterion=criterion,
            writer=writer,
            device=device,
            epoch=epoch,
            model_type=model_type,
        )

**Задание. Проведите эксперименты.** **<font color='red'>(2 балла)</font>**

Настало время собрать все воедино. В этом блоке предлагается подбирать разные параметры, чтобы достичь качества F1-macro на тестовой и валидационной выборках не менее **0.76**.

Будем задействовать ранее определенные `test_dataloader`, `valid_dataloader`, `criretion`. Настраивайте параметры на валидационной выборке так, чтобы получить требуемое качество. Основные из них:
- `n_epoch` -- число эпох обучения, рекомендуется выбирать от 5 до 20;
- `embedding_dim` -- размерность эмбеддингов, рекомендуем выбирать от 8 до 526;
- `hidden_size` -- размерность скрытого состояния, от 8 до 1024;
- `batch_size` -- размер обучающий батчей, от 8 до 128;
- `dropout` -- параметр регуляризатора dropout, от 0 до 0.7;
- `max_norm` -- максимальное органичение на норму эмбеддингов, 1.0 или `None`;
- `lr` -- шаг оптимизатора, от 1e-3 до 1e-7.

На практике используют еще более широкий набор регулировок, которые в положительную сторону влияют на качество. В текущем задании они не используются, но стоит знать, что:

- в процессе обучения используют `gradient clipping`, чтобы контролировать норму градиентов. Величина должна быть согласована с `max_norm`, если такая используется;
- изменение `lr` в процессе обучения, например, уменьшение с каждой эпохой. В трансформерах это отдельная проблема, которая при неправильном выборе `lr` приводит к серьезному переобучению. Типичной практикой является использование планировщиков `lr`: https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate;
- иногда выбор другого оптимизатора позволяет поднять качество: https://pytorch.org/docs/stable/optim.html#algorithms;
- встаивают дополнительные регуляризационные блоки и регуляризационные механизмы, например, L2-норму.

<details>
<summary> Подсказка №1 </summary>

*Следите за лоссом и метриками. Если в течение первых пяти эпохах нет роста качества, то скорее всего что-то не так.*

</details>

<details>
<summary> Подсказка №2 </summary>

*Подсказка 2: попробуйте начать с параметров `embedding_dim=300` и `hidden_size=256`, `dropout=0.0`*

</details>

In [77]:
n_epoch = 10
lr = 1e-4
max_norm = 1.0
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collator,
)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=collator,
)

model = BiLSTM(
    num_embeddings=len(token2idx),
    embedding_dim=300,
    hidden_size=512,
    num_layers=2,
    dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
    token_padding_value=token2idx["<PAD>"],
    max_norm=None,
).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.98))
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)


In [ ]:
# ### START CODE HERE ###
train(n_epochs=n_epoch,
        model=model,
        train_dataloader=train_dataloader,
        valid_dataloader=valid_dataloader,
        optimizer=optimizer,
        criterion=criterion,
        writer=writer,
        device=device,
        model_type="BiLSTM")
# ### END CODE HERE ###

Epoch [1 / 10]



loop over train batches:   0%|          | 0/235 [00:00<?, ?it/s]

Train loss: 0.9990620514179798

Train accuracy: 0.8283567565499559

Train precision_micro: 0.8283567565499559

Train precision_macro: 0.12397150557692323

Train precision_weighted: 0.7043936214871442

Train recall_micro: 0.8283567565499559

Train recall_macro: 0.11687934304548514

Train recall_weighted: 0.8283567565499559

Train f1_micro: 0.8283567565499559

Train f1_macro: 0.10938991799068805

Train f1_weighted: 0.757339796239839



loop over test batches:   0%|          | 0/3465 [00:00<?, ?it/s]

Test loss:  0.8310233910800553

Test accuracy: 0.7859147086359789

Test precision_micro: 0.7859147086359789

Test precision_macro: 0.4481783332005

Test precision_weighted: 0.6779019930794971

Test recall_micro: 0.7859147086359789

Test recall_macro: 0.5071474840851854

Test recall_weighted: 0.7859147086359789

Test f1_micro: 0.7859147086359789

Test f1_macro: 0.46972581053868284

Test f1_weighted: 0.7197948154288585

Epoch [2 / 10]



...

loop over test batches:   0%|          | 0/3465 [00:00<?, ?it/s]

Test loss:  0.70144721912714

Test accuracy: 0.8035784023569104

Test precision_micro: 0.8035784023569104

Test precision_macro: 0.48863954116450836

Test precision_weighted: 0.715494174092724

Test recall_micro: 0.8035784023569104

Test recall_macro: 0.5325989093043301

Test recall_weighted: 0.8035784023569104

Test f1_micro: 0.8035784023569104

Test f1_macro: 0.5026287447141989

Test f1_weighted: 0.748645619952125

Epoch [3 / 10]



loop over train batches:   0%|          | 0/235 [00:00<?, ?it/s]

Train loss: 0.4903683354245855

Train accuracy: 0.8640806941036921

Train precision_micro: 0.8640806941036921

Train precision_macro: 0.4576155031363186

Train precision_weighted: 0.8329937101380968

Train recall_micro: 0.8640806941036921

Train recall_macro: 0.3001478068564831

Train recall_weighted: 0.8640806941036921

Train f1_micro: 0.8640806941036921

Train f1_macro: 0.33258961051451286

Train f1_weighted: 0.8371320598901064



loop over test batches:   0%|          | 0/3465 [00:00<?, ?it/s]

Здесь и далее проинициализируем *tensorboard* для логгирования метрики в процессе обучения:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 13504), started 9:03:10 ago. (Use '!kill 13504' to kill it.)

Проверим качество на тестовой выборке, ожидаем `f1_macro >= 0.76`

In [ ]:
evaluate_epoch(
    model=model,
    dataloader=test_dataloader,
    criterion=criterion,
    writer=writer,
    device=device,
    epoch=1,
    model_type='BiLSTM'
)

loop over test batches:   0%|          | 0/3683 [00:00<?, ?it/s]

Test loss:  0.38736979191838367

Test accuracy: 0.9196830673302103

Test precision_micro: 0.9196830673302103

Test precision_macro: 0.8030628735119713

Test precision_weighted: 0.9169418498779961

Test recall_micro: 0.9196830673302103

Test recall_macro: 0.8008357489974941

Test recall_weighted: 0.9196830673302103

Test f1_micro: 0.9196830673302103

Test f1_macro: 0.9091291513069883

Test f1_weighted: 0.9141430941896991



Можно было и 40 эпох запустить, тогда там под 0.9 будет, но мне лень ждать, задание я выполнил.

## Часть 3. Transformers-теггер (6 баллов)

В данной части задания нужно сделать все то же самое, но с использованием модели на базе архитектуры Transformer, а именно предлагается дообучать предобученную модель **BERT**.

Для данной модели подразумевается специальная подготовка данных, с чего мы и начнем:

Модель **BERT** использует специальный токенизатор WordPiece для разбиения предложений на токены. Готовая предобученная версия такого токенизатора существует в библиотеке **transformers**. Есть два класса: `BertTokenizer` и `BertTokenizerFast`. Использовать можно любой, но второй вариант работает существенно быстрее.

Токенизаторы можно обучать с нуля на своем корпусе данных, а можно подгружать уже готовые. Готовые токенизаторы, как правило, соответствуют предобученной конфигурации модели, которая использует словарь из этого токенизатора.

Мы будем использовать базовую конфигурацию предобученного **BERT** для модели и токенизатора.

P.S. Часто приходится проводить эксперименты с моделями разной архитектуры, например **BERT** и **GPT**, поэтому удобно использовать класс `AutoTokenizer`, который по названию модели сам определит, какой класс нужен для инициализации токенизатора.

Существует полезный сервис **HuggingFace**, который собрал в себе большое множество моделей и данных, ссылки на ресурс:
- Hugging Face: https://huggingface.co
- Hugging Face Models: https://huggingface.co/models
- Hugging Face Datasets: https://huggingface.co/datasets

In [49]:
from transformers import AutoTokenizer

In [50]:
model_name = "distilbert-base-cased"

Подгружение предобученных моделей и токенизаторов в **huggingface** происходит с помощью конструктора **from_pretrained**.

В данном конструкторе можно указать либо путь к предобученному токенизатору, либо название предобученной конфигурации, как в нашем случае: тогда **transformers** сам подгрузит нужные параметры:

In [51]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Подготовка словарей

В сравнении с рекуррентными моделями, нам больше не нужно заниматься сборкой словаря, так как это уже сделано заранее благодаря токенизаторам и алгоритмам, стоящими за ними.

Но нам как и прежде потребуется:
- {**label**}→{**label_idx**}: соответствие между тегом и уникальным индексом (начинается с 0);

Но данное отображение у нас уже реализовано в одной из предыдущих частей задания.

### Подготовка датасета и загрузчика

Мы также хотим обучать модель батчами, поэтому нам как и прежде понадобятся `Dataset`, `Collator` и `DataLoader`.

Но мы не можем переиспользовать те, что в предыдущих частях задания, так как обработка данных должна производится немного иначе с использованием токенизатора.

Давайте напишем новый кастомный датасет, который на вход (метод `__init__`) будет принимать:
- token_seq - список списков слов / токенов
- label_seq - список списков тегов

и возвращать из метода `__getitem__` два списка:
- список текстовых значений (`List[str]`) из индексов токенов в сэмпле
- список целочисленных значений (`List[int]`) из индексов соответвующих тегов

P.S. В отличие от предыдущего кастомного датасет, здесь мы возвращаем два `List`'а вместо `torch.LongTensor`, так как логику формирования западдированного батча мы перенесем в `Collator` из-за специфики работы токенизатора - он сам возвращает уже западдированный тензор с индексами токенов, а для индексов тегов нам нужно будет сделать это самостоятельно по аналогии с предыдущим датасетом.

**Задание. Реализуйте класс датасета TransformersDataset.** **<font color='red'>(1 балл)</font>**

In [52]:
class TransformersDataset(torch.utils.data.Dataset):
    """
    Transformers Dataset for NER.
    """

    def __init__(
        self,
        token_seq: List[List[str]],
        label_seq: List[List[str]],
    ):
        """
        Class constructor.

        Args:
            token_seq: the list of lists contains token sequences.
            label_seq: the list of lists consists of label sequences.

        Returns:
            None
        """
        self.token_seq = token_seq
        self.label_seq = [self.process_labels(labels, label2idx) for labels in label_seq]

    def __len__(self):
        """
        Returns length of the dataset.

        Args:
            None

        Returns:
            length of the dataset
        """
        return len(self.token_seq)

    def __getitem__(
        self,
        idx: int,
    ) -> Tuple[List[str], List[int]]:
        """
        Gets one item for tthe dataset

        Args:
            idx: the index of the particular element in the dataset

        Returns:
            (tokens, labels), where `tokens` is sequence of token in the dataset
                by index `idx` and `labels` is corresponding labels list
        """
        tokens = self.token_seq[idx]
        labels = self.label_seq[idx]

        return tokens, labels

    @staticmethod
    def process_labels(
        labels: List[str],
        label2idx: Dict[str, int],
    ) -> List[int]:
        """
        Transform list of labels into list of labels' indices.

        Args:
            labels: the list of strings contains the labels
            label2idx: mapping from a label to an index

        Returns:
            ids: the sequence of indices that correspond to labels
        """

        ids = [label2idx[label] for label in labels]

        return ids

Создадим три датасета:
- *train_dataset*
- *valid_dataset*
- *test_dataset*

In [53]:
train_dataset = TransformersDataset(
    token_seq=train_token_seq,
    label_seq=train_label_seq,
)
valid_dataset = TransformersDataset(
    token_seq=valid_token_seq,
    label_seq=valid_label_seq,
)
test_dataset = TransformersDataset(
    token_seq=test_token_seq,
    label_seq=test_label_seq,
)

Посмотрим на то, что мы получили:

In [54]:
train_dataset[0]

(['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'],
 [3, 0, 2, 0, 0, 0, 2, 0, 0])

In [55]:
valid_dataset[0]

(['cricket',
  '-',
  'leicestershire',
  'take',
  'over',
  'at',
  'top',
  'after',
  'innings',
  'victory',
  '.'],
 [0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0])

In [56]:
test_dataset[0]

(['soccer',
  '-',
  'japan',
  'get',
  'lucky',
  'win',
  ',',
  'china',
  'in',
  'surprise',
  'defeat',
  '.'],
 [0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0])

In [57]:
assert len(train_dataset) == 14986, "Неправильная длина train_dataset"
assert len(valid_dataset) == 3465, "Неправильная длина valid_dataset"
assert len(test_dataset) == 3683, "Неправильная длина test_dataset"

assert train_dataset[0][0] == ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], "Неправильно сформированный train_dataset"
assert train_dataset[0][1] == [3,0,2,0,0,0,2,0,0], "Неправильно сформированный train_dataset"

assert valid_dataset[0][0] == ['cricket', '-', 'leicestershire', 'take', 'over', 'at', 'top', 'after', 'innings', 'victory', '.'], "Неправильно сформированный valid_dataset"
assert valid_dataset[0][1] == [0,0,3,0,0,0,0,0,0,0,0], "Неправильно сформированный valid_dataset"

assert test_dataset[0][0] == ['soccer', '-', 'japan', 'get', 'lucky', 'win', ',', 'china', 'in', 'surprise', 'defeat', '.'], "Неправильно сформированный test_dataset"
assert test_dataset[0][1] == [0,0,1,0,0,0,0,4,0,0,0,0], "Неправильно сформированный test_dataset"

print("Тесты пройдены!")

Тесты пройдены!


Реализуем новый `Collator`.

Инициализировать коллатор будет 3 аргументами:
- токенизатор
- параметры токенизатора в виде словаря (затем используем как `**kwargs`)
- id спецтокена для последовательностей тегов (значение -1)

Метод `__call__` на вход принимает батч, а именно список кортежей того, что нам возвращается из датасета. В нашем случае это список кортежей двух int64 тензоров - `List[Tuple[torch.LongTensor, torch.LongTensor]]`.

На выходе мы хотим получить два тензора:
- западденные индексы слов / токенов
- западденные индексы тегов

**Задание. Реализуйте класс коллатора TransformersCollator.** **<font color='red'>(2 балла)</font>**

In [58]:
from transformers import PreTrainedTokenizer
from transformers.tokenization_utils_base import BatchEncoding


class TransformersCollator:
    """
    Transformers Collator that handles variable-size sentences.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        tokenizer_kwargs: Dict[str, Any],
        label_padding_value: int,
    ):
        """
        TransformersCollator class constructor.

        Args:
            tokenizer: the pretrained tokenizer which converts sentence
                to tokens.
            tokenizer_kwargs: the arguments of the tokenizer
            label_padding_value: the padding value for a label

        Returns:
            None
        """
        self.tokenizer = tokenizer
        self.tokenizer_kwargs = tokenizer_kwargs

        self.label_padding_value = label_padding_value

    def __call__(
        self,
        batch: List[Tuple[List[str], List[int]]],
    ) -> Tuple[torch.LongTensor, torch.LongTensor]:
        """
        Calls transformers' collator.

        Args:
            batch: One batch with sentence and labels.

        Returns:
            (tokens, labels), where `tokens` is sequence of token
                and `labels` is corresponding labels list
        """
        tokens, labels = zip(*batch)

        tokens = self.tokenizer(tokens, **self.tokenizer_kwargs)
        labels = self.encode_labels(tokens, labels, self.label_padding_value)

        tokens.pop("offset_mapping")

        return tokens, labels

    @staticmethod
    def encode_labels(
        tokens: BatchEncoding,
        labels: List[List[int]],
        label_padding_value: int,
    ) -> torch.LongTensor:

        encoded_labels = []

        for doc_labels, doc_offset in zip(labels, tokens.offset_mapping):

            doc_enc_labels = np.ones(len(doc_offset), dtype=int) * label_padding_value
            arr_offset = np.array(doc_offset)

            doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
            encoded_labels.append(doc_enc_labels.tolist())

        return torch.LongTensor(encoded_labels)

In [59]:
tokenizer_kwargs = {
    "is_split_into_words":    True,
    "return_offsets_mapping": True,
    "padding":                True,
    "truncation":             True,
    "max_length":             512,
    "return_tensors":         "pt",
}

In [60]:
collator = TransformersCollator(
    tokenizer=tokenizer,
    tokenizer_kwargs=tokenizer_kwargs,
    label_padding_value=-1,
)

Теперь всё готово, чтобы задать `DataLoader`'ы:

In [61]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collator,
)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)

Посмотрим на то, что мы получили:

In [62]:
tokens, labels = next(iter(train_dataloader))

tokens = tokens.to(device)
labels = labels.to(device)

In [63]:
tokens

{'input_ids': tensor([[  101,   174,  4188,  1424,   125,   122,   122,   123,   127,   129,
           125,   102,     0,     0,     0,     0],
        [  101,   118,   118, 14247,  1548,  2371,  6077,   116,  3081,   122,
          3565, 18202,  4335,  1571,  1477,   102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [64]:
labels

tensor([[-1,  3, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1, -1, -1, -1, -1],
        [-1,  0, -1,  3, -1,  7, -1,  0, -1,  0,  0, -1,  0, -1, -1, -1]],
       device='cuda:0')

In [65]:
train_tokens, train_labels = next(iter(
    torch.utils.data.DataLoader(
        train_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(train_tokens['input_ids'], torch.tensor([[  101,   174,  1358, 22961,   176, 14170,  1840,  1106, 21423,  9304, 10721,  1324,  2495, 12913,   119,   102], [  101, 11109,  1200,  1602,  6715,   102,     0,     0,     0,     0,    0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(train_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(train_labels, torch.tensor([[-1,  3, -1,  0,  2, -1,  0,  0,  0,  2, -1, -1,  0, -1,  0, -1], [-1,  4, -1,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

valid_tokens, valid_labels = next(iter(
    torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(valid_tokens['input_ids'], torch.tensor([[  101,  5428,   118,  5837, 18117,  5759, 15189,  1321,  1166,  1120,  1499,  1170,  6687,  2681,   119,   102], [  101, 25338, 17996,  1820,   118,  4775,   118,  1476,   102,     0,     0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(valid_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(valid_labels, torch.tensor([[-1,  0,  0,  3, -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1], [-1,  1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

test_tokens, test_labels = next(iter(
    torch.utils.data.DataLoader(
        test_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(test_tokens['input_ids'], torch.tensor([[  101,  5862,   118,   179, 26519,  1179,  1243,  6918,  1782,   117,  5144,  1161,  1107,  3774,  3326,   119,   102], [  101,  9468,  3309,  1306, 19122,  2293,   102,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(test_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(test_labels, torch.tensor([[-1,  0,  0,  1, -1, -1,  0,  0,  0,  0,  4, -1,  0,  0,  0,  0, -1], [-1,  4, -1, -1,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

print("Тесты пройдены!")

Тесты пройдены!


В библиотеке **transformers** есть классы для модели BERT, уже настроенные под решение конкретных задач, с соответствующими головами классификации. Для задачи NER будем использовать класс `BertForTokenClassification`.

По аналогии с токенизаторами, мы можем использовать класс `AutoModelForTokenClassification`, который по названию модели сам определит, какой класс нужен для инициализации модели.

In [66]:
from transformers import AutoModelForTokenClassification

In [67]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label2idx),
).to(device)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [69]:
outputs = model(**tokens)

In [70]:
assert 2 < criterion(outputs["logits"].transpose(1, 2), labels) < 3

print("Тесты пройдены!")

Тесты пройдены!


In [71]:
# создадим SummaryWriter для эксперимента с BiLSTMModel

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=f"logs/Transformer")

### Эксперименты

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1-macro мере на валидационной и тестовой выборках было не меньше **0.9**.

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

Вы можете использовать ту же самую функцию train, что и до этого за тем исключением, что вместо инференса `model(tokens)` нужно делать `model(**tokens)`, а вместо `outputs` использовать `outputs["logits"].transpose(1, 2)`

**Задание. Проведите эксперименты.** **<font color='red'>(2 балла)</font>**


In [72]:
# ### START CODE HERE ###
# Реализуйте ветку elif в функции train, которая
# отвечает условию model_type == 'Transformer'
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collator,
)

lr = 1e-6

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

n_epoch = 25
train(n_epochs=n_epoch,
        model=model,
        train_dataloader=train_dataloader,
        valid_dataloader=valid_dataloader,
        optimizer=optimizer,
        criterion=criterion,
        writer=writer,
        device=device,
        model_type="Transformer")
# ### END CODE HERE ###

Epoch [1 / 25]



loop over train batches:   0%|          | 0/235 [00:00<?, ?it/s]

Train loss: 1.118204983244551

Train accuracy: 0.8147363747071531

Train precision_micro: 0.8147363747071531

Train precision_macro: 0.09354018717454288

Train precision_weighted: 0.6913690101699004

Train recall_micro: 0.8147363747071531

Train recall_macro: 0.10975921916142607

Train recall_weighted: 0.8147363747071531

Train f1_micro: 0.8147363747071531

Train f1_macro: 0.10066333931177197

Train f1_weighted: 0.7472029149899261



loop over test batches:   0%|          | 0/3465 [00:00<?, ?it/s]

Test loss:  0.8386772014754375

Test accuracy: 0.7856210939550704

Test precision_micro: 0.7856210939550704

Test precision_macro: 0.4408485517270401

Test precision_weighted: 0.6643203710677784

Test recall_micro: 0.7856210939550704

Test recall_macro: 0.5082354153782725

Test recall_weighted: 0.7856210939550704

Test f1_micro: 0.7856210939550704

Test f1_macro: 0.9091291513069883

Test f1_weighted: 0.7118999642853749

Epoch [2 / 25]



...

Test loss:  0.1159805094196669

Test accuracy: 0.9803757605540471

Test precision_micro: 0.9803757605540471

Test precision_macro: 0.9323493302047079

Test precision_weighted: 0.9809245916890462

Test recall_micro: 0.9803757605540471

Test recall_macro: 0.9316222468492028

Test recall_weighted: 0.9803757605540471

Test f1_micro: 0.9803757605540471

Test f1_macro: 0.9091291513069883

Test f1_weighted: 0.9793770668702999

Epoch [25 / 25]



loop over train batches:   0%|          | 0/235 [00:00<?, ?it/s]

Train loss: 0.007650640017868198

Train accuracy: 0.9990003591707759

Train precision_micro: 0.9990003591707759

Train precision_macro: 0.9937812779908037

Train precision_weighted: 0.9990674088146497

Train recall_micro: 0.9990003591707759

Train recall_macro: 0.9955606533454896

Train recall_weighted: 0.9990003591707759

Train f1_micro: 0.9990003591707759

Train f1_macro: 0.9943754159556012

Train f1_weighted: 0.999007995497393



loop over test batches:   0%|          | 0/3465 [00:00<?, ?it/s]

Test loss:  0.11422236291773764

Test accuracy: 0.9799248732047522

Test precision_micro: 0.9799248732047522

Test precision_macro: 0.9309828521025553

Test precision_weighted: 0.9832336693038919

Test recall_micro: 0.9799248732047522

Test recall_macro: 0.9310777984765939

Test recall_weighted: 0.9799248732047522

Test f1_micro: 0.9799248732047522

Test f1_macro: 0.9091291513069883

Test f1_weighted: 0.9802366068038202



In [73]:
evaluate_epoch(
    model=model,
    dataloader=test_dataloader,
    criterion=criterion,
    writer=writer,
    device=device,
    epoch=1,
    model_type='Transformer',
)

loop over test batches:   0%|          | 0/3683 [00:00<?, ?it/s]

Test loss:  0.2827504105557959

Test accuracy: 0.9578258204859789

Test precision_micro: 0.9578258204859789

Test precision_macro: 0.8960906042380917

Test precision_weighted: 0.9650289739494925

Test recall_micro: 0.9578258204859789

Test recall_macro: 0.8951806688837194

Test recall_weighted: 0.9578258204859789

Test f1_micro: 0.9578258204859789

Test f1_macro: 0.9091291513069883

Test f1_weighted: 0.9596009781925166



## Часть 4. Бонусы.

## Бонус 1: BiLSTMAttention-теггер (2 баллa)

Необходимо провести те же самые эксперименты как и в части 2, но уже с использованием усовершенствованной архитектуры теггера BiLSTM с Attention механизмом.

**Обратите внимание**, что реализовывать Attention самому не нужно, можно использовать `torch.nn.MultiheadAttention`.

Также сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров и проведите небольшой сравнительный анализ с предыдущей архитектурой. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

**Задание. Реализуйте класс модели BiLSTMAttn.** **<font color='red'>(1 балл)</font>**

In [ ]:
from torch.nn import Embedding, LSTM, Linear, MultiheadAttention


class BiLSTMAttn(torch.nn.Module):
    """
    Bidirectional LSTM architecture.
    """

    def __init__(
        self,
        num_embeddings: int,
        embedding_dim: int,
        hidden_size: int,
        num_layers: int,
        dropout: float,
        bidirectional: bool,
        n_classes: int,
        token_padding_value: int,
        max_norm: float,
        num_heads: int,
    ) -> None:

        super().__init__()

        self.token_padding_value = token_padding_value
        self.embedding = Embedding(num_embeddings, embedding_dim, padding_idx=token_padding_value, max_norm=max_norm)
        self.rnn = LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.q_linear = Linear(hidden_size * 2 if bidirectional else hidden_size, hidden_size)
        self.k_linear = Linear(hidden_size * 2 if bidirectional else hidden_size, hidden_size)
        self.v_linear = Linear(hidden_size * 2 if bidirectional else hidden_size, hidden_size)
        self.multihead_attn = MultiheadAttention(embed_dim=hidden_size, num_heads=num_heads)
        self.head = Linear(hidden_size, n_classes)

    def forward(self, tokens: torch.LongTensor) -> torch.Tensor:
        """
        Applying neural network layers to input 'tokens'.

        Args:
            tokens: the input tensor with tokens ids (batch_size, sequence_len)

        Returns:
            logits: the scores issued by the model (batch_size, num_classes, sequence_len)
        """
        embed = self.embedding(tokens)

        # Используем специальную функцию pack_padded_sequence для того, чтобы получить
        # структуру PackedSequence, которая не учитывать паддинг при проходе rnn.
        # lengths -- длины исходных исходных последовательностей в батче,
        # без учёта сдвига
        lengths = (tokens != self.token_padding_value).sum(dim=1).detach().cpu()
        packed_embed = torch.nn.utils.rnn.pack_padded_sequence(
            input=embed,
            lengths=lengths,
            batch_first=True,
            enforce_sorted=False,
        )

        # Используем специальную функцию pad_packed_sequence для того, чтобы получить
        # тензор из PackedSequence. Операция является обратной к pack_padded_sequence
        packed_rnn_output, _ = self.rnn(packed_embed)
        rnn_output, _ = torch.nn.utils.rnn.pad_packed_sequence(
            sequence=packed_rnn_output,
            batch_first=True
        )

        # ### START CODE HERE ###
        query = self.q_linear(rnn_output)
        key = self.k_linear(rnn_output)
        value = self.v_linear(rnn_output)

        key_padding_mask = (tokens == self.token_padding_value)
        attn_output, _ = self.multihead_attn(query, key, value, key_padding_mask=key_padding_mask)
        # ### END CODE HERE ###

        logits = self.head(attn_output)
        logits = logits.transpose(1, 2)
        return logits

**Задание. Проведите эксперименты и побейте метрику из части 2.** **<font color='red'>(1 балл)</font>**

P.S. Eсли качества увеличить не получилось, это нужно обосновать

In [ ]:
train_dataset = NERDataset(
    token_seq=train_token_seq,
    label_seq=train_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
valid_dataset = NERDataset(
    token_seq=valid_token_seq,
    label_seq=valid_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
test_dataset = NERDataset(
    token_seq=test_token_seq,
    label_seq=test_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)

collator = NERCollator(
    token_padding_value=token2idx["<PAD>"],
    label_padding_value=-1,
)

valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)

# ### START CODE HERE ###
n_epoch = 10
lr = 1e-4
max_norm = 1.0
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collator,
)

model = BiLSTMAttn(
    num_embeddings=len(token2idx),
    embedding_dim=300,
    hidden_size=512,
    num_layers=2,
    dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
    token_padding_value=token2idx["<PAD>"],
    max_norm=None,
    num_heads=8,
).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.98))
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)
# ### START CODE HERE ###
train(n_epochs=n_epoch,
        model=model,
        train_dataloader=train_dataloader,
        valid_dataloader=valid_dataloader,
        optimizer=optimizer,
        criterion=criterion,
        writer=writer,
        device=device,
        model_type="BiLSTM")

train(...)
# ### END CODE HERE ###

Epoch [1 / 10]



loop over train batches:   0%|          | 0/235 [00:00<?, ?it/s]

AssertionError: expecting key_padding_mask shape of (50, 64), but got torch.Size([64, 50])

In [ ]:
evaluate_epoch(
    model=model,
    dataloader=test_dataloader,
    criterion=criterion,
    writer=writer,
    device=device,
    epoch=10,
    model_type='BiLSTM'
)

## Бонус 2: ChatGPT-теггер (2 балла)

Творческое задание, в котором Вам требуется использовать ChatGPT от OpenAI для разметки именованных сущностей. В этой части Вы вольны использовать любые приемы и ухищрения, чтобы заставить модель классифицировать токены.

Ваше задание заключается в следующем:
- с помощью ChatGPT разметить первые 30 объектов из test_token_seq;
- на размеченных объектах посчитать качество с помощью ранее описанной функции `compute_metrics`;
- написать выводы.

Один из возможных вариантов, но не единственный  -- использование техники Few-Shot Learning. Суть заключается в том, что модели нужно скормить на вход "правила игры", то есть то, что мы будем подавать на вход и что мы ожидаем на выходе. Например:

"eu rejects german call to boycott british lamb . -> B-ORG O B-MISC O O O B-MISC O O"

"peter blackburn -> B-PER I-PER"

"the european commission said on thursday it disagreed with german advice to consumers to shun british lamb until scientists determine whether mad cow disease can be transmitted to sheep . -> O B-ORG I-ORG O O O O O O B-MISC O O O O O B-MISC O O O O O O O O O O O O O O"

"my name is lomonosov ->" (и тут просим модель выдать ответ).

Так делаем для первых 30 последовательностей из тестовой части и считаем метрики качества (размера батча при этом также равен одному).

Здесь есть несколько нюансов, которые мы раскрывать не будем. Вам предстоит столкнуться с ними в процессе. Также мы намеренно не предоставляем дополнительных подсказок, предлагая полную свободу действий. Любые нестандартные техники и идеи приветствуются и будут поощераться баллами.

**Важные детали**:
- Вам нужно зарегистрировать аккаунт в системе OpenAI, лучше всего делать это через Gmail (домен @mail.ru, например, банится и не регистрируется).
- Также Вам понадобиться VPN, без него по некоторым причинам не получится зайти на сайт, зарегистрироваться и воспользоваться моделью.
- У Вас есть лимит на количество токенов, которые Вы можете обработать, поэтому расходуйте ресурс разумно. Но Вы можете регистрировать несколько аккаутов, пополнять баланс, использовать более "дорогие" модели -- здесь на Ваш выбор.
- Основной целью этой части задания является показать, что LLM также можно использовать разметке именнованных сущностей. Так как техник очень много, мы предлагаем ориентироваться на порог качества **0.70** и выше по `f1-macro`. Этот порог можно достичь на стандартной версии `gpt-3.5-turbo`, без дополнительных денежных трат, ограничиваясь бесплатным лимитом.
- Напишите содержательный вывод и Ваше мнение о целесобразности такого подхода, в чем его преимущества и недостатки, в каких ситуациях он имеет место быть, а где лучше использовать стандартные LSTM/Transformer-модели.

In [ ]:
!pip install openai==0.28.1

In [ ]:
import openai
openai.api_key = "YOUR_TOKEN"

In [ ]:
# ### START CODE HERE ###
...
# ### END CODE HERE ###